# Analysis of Ski Resort Data

## Summary

This notebook explores the data taken from www.skiresorts.info using a selenium based scraper, see https://github.com/awkirby/ski-scraping.git.

The data was partially cleaned post-scraping, Pandas and other tools (plotly, pyplot etc.) are used to complete the cleaning and visualise the data set.

The statsmodel module will then be used to fit a linear regression based model to the data in order to calculate the cost of a ski pass at each resort. 

A cost estimator could be a useful tool either for the resorts themselves or for customers to identify whether a resort is good value for money or not.

In [12]:
import pandas as pd
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 200)

import numpy as np

In [13]:
# Path to csv file
path = "results/ski_resort_data_clean.csv"

# Lets start by looking at what we have
df_ski_data = pd.read_csv(path)

# Get some summary statistics
df_ski_data.describe()

,Unnamed: 0,Unnamed: 0.1,Access Order,ID,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Cost in Euros
count,3397.000000,3397.000000,3397.000000,3397.000000,3049.000000,3397.000000,3397.000000,3397.000000,3397.000000,3378.000000,3378.000000,3378.000000,3397.000000,3397.000000,3397.000000
mean,2045.925817,2045.925817,2046.925817,7359.718281,2.546146,367.386223,850.466294,1217.852517,16.178246,6.124719,6.957608,3.123653,5.595525,810.317592,31.119974
std,1316.264607,1316.264607,1316.264607,9811.071014,0.618926,371.116761,551.851679,774.428130,34.955345,14.429009,16.088254,8.214040,8.261449,4614.145507,21.298469
min,0.000000,0.000000,1.000000,1.000000,1.700000,5.000000,3.000000,25.000000,0.100000,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000
25%,917.000000,917.000000,918.000000,1777.000000,2.000000,102.000000,456.000000,647.000000,1.500000,0.800000,0.500000,0.000000,2.000000,23.000000,17.000000
50%,1908.000000,1908.000000,1909.000000,3506.000000,2.400000,225.000000,764.000000,1025.000000,5.000000,2.000000,2.000000,0.500000,3.000000,44.000000,27.000000
75%,3107.000000,3107.000000,3108.000000,5929.000000,2.900000,504.000000,1195.000000,1740.000000,15.000000,5.475000,6.200000,2.600000,6.000000,175.000000,39.000000
max,5873.000000,5873.000000,5874.000000,34625.000000,4.900000,2509.000000,3290.000000,3950.000000,600.000000,312.000000,238.500000,126.000000,170.000000,85000.000000,350.000000


In [14]:
# Get some info
df_ski_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               3397 non-null   int64  
 1   Unnamed: 0.1             3397 non-null   int64  
 2   Access Order             3397 non-null   int64  
 3   ID                       3397 non-null   int64  
 4   Name                     3397 non-null   object 
 5   Continent                3397 non-null   object 
 6   Country                  3396 non-null   object 
 7   Web Link                 3397 non-null   object 
 8   Star Rating              3049 non-null   float64
 9   Elevation Change (m)     3397 non-null   int64  
 10  Base Elevation (m)       3397 non-null   int64  
 11  Max Elevation (m)        3397 non-null   int64  
 12  Total Piste Length (km)  3397 non-null   float64
 13  Blue Piste Length (km)   3378 non-null   float64
 14  Red Piste Length (km)   

We observe the following:

* There are several redundant columns probably due to the index being stored each time the data was saved and reloaded
* The following columns contain null values: Country, Star Rating, Blue/Red/Black Psite Lengths, and Photo
* Pandas lists stings as dtype "object" 

Our aim is to use resort information to estimate the price of a ski pass, therefore columns with information on the photos or links to the source data (i.e. "Web Link") are also redundant. 


In [15]:
# The most important features have been identified in a Table.
# These will inform the process.
feature_assessment = pd.read_excel("variable_comments.xlsx")
feature_assessment

,Variable Name,Type,Expectation,Include,Comments
0,ID,Categorical,5,N,Purely related to internal website database
1,Name,Categorical,5,N,All names are unique
2,Continent,Categorical,3,Y,"The best resorts are in Europe and North America, usually in developed countries. I expect this to be reflected in the price."
3,Country,Categorical,2,Y,"As above, but greater impact as it should be easier to identify a trend."
4,Web Link,Categorical,5,N,Unique and unrelated to resort
5,Star Rating,Numerical,2,Y,More popular or better resorts should have higher prices.
6,Elevation Change,Numerical,3,Y,Greater elevation change leads to a more enjoyable experience
7,Base Elevation,Numerical,4,Y,"Higher altitude increases cost of operation, but decreases cost of snow management"
8,Max Elevation,Numerical,4,N,"As with base elevation, but don't include since it can be derived from the previous two variables"
9,Total Piste Length,Numerical,2,Y,Plays a key role in the value of a resort


## Data Cleaning

In [16]:
# Drop redundant columns
redundant = ["Unnamed: 0", "Unnamed: 0.1", "Web Link", "Photo URL", "Photo", "Page Link"]

df_ski_data.drop(columns=redundant, inplace=True)

# Review
df_ski_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Access Order             3397 non-null   int64  
 1   ID                       3397 non-null   int64  
 2   Name                     3397 non-null   object 
 3   Continent                3397 non-null   object 
 4   Country                  3396 non-null   object 
 5   Star Rating              3049 non-null   float64
 6   Elevation Change (m)     3397 non-null   int64  
 7   Base Elevation (m)       3397 non-null   int64  
 8   Max Elevation (m)        3397 non-null   int64  
 9   Total Piste Length (km)  3397 non-null   float64
 10  Blue Piste Length (km)   3378 non-null   float64
 11  Red Piste Length (km)    3378 non-null   float64
 12  Black Piste Length (km)  3378 non-null   float64
 13  Ski Lifts                3397 non-null   int64  
 14  Ski Pass Cost           

In [17]:
# The access order values are unique, use these as the index instead
df_ski_data.set_index('Access Order', inplace=True)

In [18]:
# Check data
df_ski_data.head()

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
1,119,Obertauern,Europe,Austria,4.5,683,1630,2313,100.0,61.0,35.0,4.0,26,49.0,€,49.0
2,107,Lermoos – Grubigstein,Europe,Austria,4.0,1096,1004,2100,27.3,12.8,12.6,1.9,8,49.5,€,49.5
3,72,Spieljoch – Fügen,Europe,Austria,3.7,1404,650,2054,17.1,3.6,10.8,2.7,7,59.0,€,59.0
4,40,Steinplatte/Winklmoosalm – Waidring/Reit im Winkl,Europe,Austria,4.1,1120,740,1860,42.0,18.0,22.0,2.0,14,50.0,€,50.0
5,158,Ski Juwel Alpbachtal Wildschönau,Europe,Austria,4.2,1195,830,2025,90.9,25.1,53.1,12.7,46,51.0,€,51.0


In [19]:
# From the .describe() of the data the maximum cost of 350 euros is well outside the 3rd Quartile price of 39 euros
# Have a look at these more expensive resorts
df_ski_data[df_ski_data["Cost in Euros"] > 100.0][["Name", "Country", "Ski Pass Cost", "Currency", "Cost in Euros"]]

,Name,Country,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,
67,Vail,USA,219.00,US$,184.0
71,Telluride,USA,169.00,US$,142.0
73,Beaver Creek,USA,218.00,US$,183.0
78,Mammoth Mountain,USA,209.00,US$,175.0
82,Snowmass,USA,199.00,US$,167.0
88,Breckenridge,USA,199.00,US$,167.0
94,Keystone,USA,176.00,US$,148.0
102,Killington,USA,165.00,US$,139.0
109,Winter Park Resort,USA,164.00,US$,138.0


Most of the more expensive resorts are in the USA! Certain areas in the US are known for their high prices so this is ok.

The two other expensive resorts are in Norway and the UK. 

The UK slope is a dry slope, so the price could be reasonable since these are expensive to run and tend to attract much smaller footfalls. However, having looked at the website, [telfordandwrekinleisure](https://www.telfordandwrekinleisure.co.uk/site/scripts/home_info.php?homepageID=20), it seems the wrong value has been used. It is the cost of learning to ski in a day, access is actually £14 for 2 hours. Given the challenge of converting this, it would be best to drop it.

The Norwegian one seems very extreme even for a country known for its high-cost of living! Looking at the resort website, https://raumaskisenter.com/priser/, it is clear that there is an error. The price should be 350 in Norwegian Kroner (NOK), equivalent to euro 35. A much more reasonable price!


In [20]:
# Correct outlier cost values
df_ski_data.drop(labels=4183, inplace=True)

df_ski_data.loc[2447,'Currency'] = "NOK"
df_ski_data.loc[2447,'Cost in Euros'] = 35.0

# Confirm
df_ski_data.loc[2447]

ID                          33350
Name                        Rauma
Continent                  Europe
Country                    Norway
Star Rating                   2.2
Elevation Change (m)          302
Base Elevation (m)            318
Max Elevation (m)             620
Total Piste Length (km)       4.1
Blue Piste Length (km)        1.4
Red Piste Length (km)         2.0
Black Piste Length (km)       0.7
Ski Lifts                       1
Ski Pass Cost               350.0
Currency                      NOK
Cost in Euros                35.0
Name: 2447, dtype: object

In [21]:
# Check the most common countries
df_ski_data['Country'].value_counts()

USA                404
Germany            392
Austria            329
Japan              278
Switzerland        272
                  ... 
Central Russia       1
Brazil               1
Lesotho              1
Portugal             1
North Macedonia      1
Name: Country, Length: 65, dtype: int64

In [22]:
# Countries with few entries may skew the results
df_ski_data['Country'].value_counts()[df_ski_data['Country'].value_counts() < 10]

Iceland                         9
China                           9
Siberia                         9
Netherlands                     7
Australia                       7
Hungary                         7
Latvia                          6
Bulgaria                        6
Georgia                         5
South Korea                     5
Belgium                         5
Serbia                          5
Turkey                          4
Lebanon                         4
Denmark                         3
Kyrgyzstan                      3
Northwest Russia                3
Andorra                         3
Estonia                         3
Southern Russia                 3
Far Eastern Federal District    2
Croatia                         2
Azerbaijan                      2
Ural Federal District           2
Montenegro                      2
Lithuania                       2
Cyprus                          1
Liechtenstein                   1
Armenia                         1
Volga Federal 

Reviewing the countries with few resorts it looks like Russian pistes have ended up in a Russian Continent with the country replaced with regions, districts etc.

## Null Values

### Country

In [23]:
# The only NaN value in the Country column is also a Russian resort
df_ski_data[df_ski_data['Country'].isnull()]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
2933,33810,Hvalin/Khvalyn,Europe,NaN,2.0,181,116,297,2.8,1.8,1.0,0.0,3,2500.0,RUB,28.0


In [24]:
# Replace NaN with Russia, otherwise the rest of the conversions won't work
df_ski_data.loc[2933,'Country'] = "Russia"
df_ski_data.loc[2933]

ID                                  33810
Name                       Hvalin/Khvalyn
Continent                          Europe
Country                            Russia
Star Rating                           2.0
Elevation Change (m)                  181
Base Elevation (m)                    116
Max Elevation (m)                     297
Total Piste Length (km)               2.8
Blue Piste Length (km)                1.8
Red Piste Length (km)                 1.0
Black Piste Length (km)               0.0
Ski Lifts                               3
Ski Pass Cost                      2500.0
Currency                              RUB
Cost in Euros                        28.0
Name: 2933, dtype: object

In [25]:
df_ski_data[df_ski_data["Continent"] == "Russia"]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
284,1571,Rosa Khutor,Russia,Southern Russia,3.6,1380,940,2320,102.0,62.0,24.0,16.0,26,3200.0,RUB,36.0
434,1569,Gazprom Mountain Resort,Russia,Southern Russia,3.2,1307,949,2256,52.8,17.3,26.3,9.2,23,3200.0,RUB,36.0
458,1568,Krasnaya Polyana Resort,Russia,Southern Russia,3.2,1212,960,2172,30.0,7.5,20.0,2.5,13,3800.0,RUB,43.0
559,5071,Zavjalikha,Russia,Ural Federal District,NaN,426,414,840,22.0,NaN,NaN,NaN,4,2600.0,RUB,30.0
684,3001,Big Wood,Russia,Northwest Russia,3.0,667,380,1047,26.7,13.5,8.5,4.7,8,1600.0,RUB,18.0
1338,1812,Baikalsk – Sobolinaya,Russia,Siberia,2.6,479,525,1004,15.0,6.6,4.9,3.5,7,2100.0,RUB,24.0
1465,1638,Adzhigardak,Russia,Ural Federal District,2.5,408,275,683,21.7,12.2,5.8,3.7,8,2500.0,RUB,28.0
1574,1863,Blagodat – Belokuricha,Russia,Siberia,2.5,523,278,801,6.1,2.3,3.2,0.6,6,1200.0,RUB,14.0
1679,33650,Manzherok,Russia,Siberia,2.4,636,388,1024,1.5,1.5,0.0,0.0,3,700.0,RUB,8.0


In [26]:
# Where the continent is Russia, set the Country entries to Russia
df_ski_data["Country"][df_ski_data["Continent"] == "Russia"] = "Russia"

<ipython-input-26-fac19350e9b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ski_data["Country"][df_ski_data["Continent"] == "Russia"] = "Russia"


In [27]:
# Where the continent is equal to "Russia", replace with "Europe"
df_ski_data["Continent"] = df_ski_data["Continent"].str.replace("Russia", "Europe")

In [28]:
# Check that the replace worked
df_ski_data["Continent"].value_counts()

Europe                   2424
North America             598
Asia                      317
Australia and Oceania      31
South America              25
Africa                      1
Name: Continent, dtype: int64

### Star Rating

Just over 10% of the Star Rating values are missing. Given the expectation that this will be a key variable in the linear regression, something must be done about this. We don't want to drop the values, since this would reduce the dataset size substantially. Using zero is not appropriate as this would impact the accuracy of the linear regression.

The safest option is to set the values equal to the current mean of **2.546146**. This could make some resorts appear better or worse than they are, but at least is shouldn't impact the overall importance of the feature.


In [29]:
# Before doing anything let's take a look
df_ski_data[df_ski_data["Star Rating"].isnull()]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
559,5071,Zavjalikha,Europe,Russia,NaN,426,414,840,22.0,NaN,NaN,NaN,4,2600.0,RUB,30.0
665,2400,Mzaar Kfardebian,Asia,Lebanon,NaN,615,1850,2465,80.0,46.0,30.0,4.0,19,71000.0,LBP,39.0
1109,2006,Bromley Mountain,North America,USA,NaN,407,594,1001,45.0,15.0,18.0,12.0,9,91.0,US$,76.0
1202,3789,Onikoube,Asia,Japan,NaN,715,340,1055,7.0,NaN,NaN,NaN,6,4000.0,¥,31.0
1237,2489,Gerlosstein,Europe,Austria,NaN,907,929,1836,10.0,1.0,8.0,1.0,5,57.5,€,57.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,3812,Osmelakowa Dolina – Spalona,Europe,Poland,NaN,50,750,800,0.4,0.4,0.0,0.0,1,55.0,PLN,12.0
4606,5896,Långberget,Europe,Sweden,NaN,40,590,630,0.5,0.5,0.0,0.0,1,125.0,Skr,12.0
4609,29363,Monkova dolina – Ždiar,Europe,Slovakia,NaN,100,850,950,0.3,0.3,0.0,0.0,1,16.0,€,16.0


In [30]:
import plotly.graph_objects as go

In [31]:
# An initial look suggests a spread of resorts, which is promising with regards to the impact
# Lets visualise using go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ski_data["Country"][df_ski_data["Star Rating"].isnull()], 
    y=df_ski_data["Cost in Euros"][df_ski_data["Star Rating"].isnull()], 
    name="Country vs Cost in Euros for Missing Star Rating Values", mode="markers"))

fig.show()

In [32]:
# Plot suggests that there are null values in a wide range of countries
# But some countries are particularly 'bad'
# Establish the percentage of missing ratings for each country
round((df_ski_data["Country"][df_ski_data["Star Rating"].isnull()].value_counts() 
       / df_ski_data["Country"].value_counts())
      *100,2)

Andorra                     NaN
Argentina                   NaN
Armenia                     NaN
Australia                 14.29
Austria                    0.91
Azerbaijan                  NaN
Belgium                   40.00
Bosnia and Herzegovina      NaN
Brazil                      NaN
Bulgaria                    NaN
Canada                    17.62
Chile                      7.14
China                     44.44
Croatia                     NaN
Cyprus                      NaN
Czech Republic            20.85
Denmark                     NaN
Estonia                   33.33
Finland                    6.67
France                     0.90
Georgia                     NaN
Germany                    0.51
Greece                      NaN
Greenland                   NaN
Hungary                     NaN
Iceland                     NaN
India                       NaN
Iran                        NaN
Israel                      NaN
Italy                      2.75
Japan                     28.42
Kazakhst

In [33]:
# Latvia and Lebanon are the only countries with less than 50% of the resorts rated
# These should be dropped to avoid skewing the results
df_ski_data[(df_ski_data["Country"] == "Latvia") | (df_ski_data["Country"] == "Lebanon")]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
665,2400,Mzaar Kfardebian,Asia,Lebanon,NaN,615,1850,2465,80.0,46.0,30.0,4.0,19,71000.0,LBP,39.0
812,5053,Zaarour,Asia,Lebanon,2.9,355,1645,2000,16.0,0.3,9.2,6.5,4,70000.0,LBP,38.0
2060,4673,The Cedars,Asia,Lebanon,NaN,755,2095,2850,9.0,4.0,2.0,3.0,7,50000.0,LBP,27.0
2315,2399,Faqra,Asia,Lebanon,NaN,240,1735,1975,7.0,2.0,4.0,1.0,4,60400.0,LBP,33.0
2466,4102,Riekstu Kalns,Europe,Latvia,2.2,36,51,87,4.9,2.5,1.8,0.6,15,29.0,€,29.0
3922,32390,Kaķīškalns,Europe,Latvia,1.9,70,25,95,0.4,0.1,0.3,0.0,2,20.0,€,20.0
4266,1814,Baiļi,Europe,Latvia,1.9,21,46,67,0.8,0.6,0.2,0.0,8,15.0,€,15.0
4316,5057,Zagarkalns,Europe,Latvia,NaN,30,70,100,1.4,1.4,0.0,0.0,12,23.0,€,23.0
4321,3408,Milzkalns,Europe,Latvia,NaN,35,75,110,1.6,1.6,0.0,0.0,8,23.0,€,23.0


In [34]:
# Get the index values for Latvian and Lebanese ski resorts
to_drop = df_ski_data[(df_ski_data["Country"] == "Latvia") | (df_ski_data["Country"] == "Lebanon")].index

# Drop these 
df_ski_data.drop(labels=to_drop, inplace=True)

In [35]:
# Check China and Belgium for any funny business
df_ski_data[(df_ski_data["Country"] == "Belgium") | (df_ski_data["Country"] == "China")]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
510,11009,Genting Resort Secret Garden,Asia,China,3.1,398,1702,2100,16.0,3.3,7.4,5.3,5,700.0,Ұ,90.0
809,1486,Duolemeidi Mountain Resort – Chongli,Asia,China,2.9,323,1640,1963,9.0,3.0,5.0,1.0,3,500.0,Ұ,65.0
1343,3217,Lianhuashan Resort,Asia,China,2.6,132,40,172,2.1,1.5,0.5,0.1,2,420.0,Ұ,54.0
1739,2756,Huaibei,Asia,China,2.4,154,146,300,4.5,1.4,3.1,0.0,3,178.0,Ұ,23.0
2136,3051,Ice Mountain (indoor ski area),Europe,Belgium,2.3,40,50,90,0.3,0.3,0.0,0.0,3,42.0,€,42.0
2546,3641,Nanshan,Asia,China,NaN,125,90,215,5.0,3.0,1.0,1.0,13,470.0,Ұ,61.0
2765,11132,Yuyang,Asia,China,NaN,220,75,295,4.0,2.0,1.0,1.0,6,448.0,Ұ,58.0
2852,2878,Jundushan,Asia,China,NaN,247,105,352,4.2,2.0,1.0,1.2,7,420.0,Ұ,54.0
2983,4813,Val de Wanne,Europe,Belgium,2.0,100,380,480,1.4,0.6,0.8,0.0,2,14.0,€,14.0


In [36]:
# For the remaining cases, and relying on the expectation that the Country feature is important,
# the average for each respective country shall be used to fill in the nan values.

# Loop through all the countries that have null values (use .unique() to avoid duplicates)
for country in df_ski_data["Country"][df_ski_data["Star Rating"].isnull()].unique():
    # Get the mean for that country
    mean_rating = df_ski_data["Star Rating"][df_ski_data["Country"] == country].mean()
    
    # Get the index of the resorts with null values in that country, this is quite a big construct so break it down
    bool_null_rating = df_ski_data["Star Rating"][df_ski_data["Country"] == country].isnull()
    null_index = df_ski_data[df_ski_data["Country"] == country][bool_null_rating].index
    
    # Replace the star rating with the mean value
    df_ski_data.loc[null_index, "Star Rating"] = mean_rating
    
# Check for null values
df_ski_data["Country"][df_ski_data["Star Rating"].isnull()].value_counts()

Series([], Name: Country, dtype: int64)

### Piste Length

There are 19 cases where the breakdown of the piste lenghts is missing. Given the small number, these could be dropped. 

However, its good to come up with solutions instead of simply dropping the values. Particularly since 13 of the resorts are in Japan, this could have a larger than necessary impact.

The suggested it approach is to set the value of red piste lengths equal to the total piste length and the other colours to 0. On average red piste's are the most common as observed when the describe method was used. 

In [37]:
# Loop through the indexes of all the resorts that are missing the piste length breakdowns
for index in df_ski_data[df_ski_data["Red Piste Length (km)"].isnull()].index:
    # Set the blue and black piste lengths to 0
    df_ski_data.loc[index, "Blue Piste Length (km)"] = 0.0
    df_ski_data.loc[index, "Black Piste Length (km)"] = 0.0
    
    # Set the Red Piste Length equal to the total piste length
    df_ski_data.loc[index, "Red Piste Length (km)"] = df_ski_data.loc[index, "Total Piste Length (km)"]

    
# Confirm there are no nulls
df_ski_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3386 entries, 1 to 5874
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       3386 non-null   int64  
 1   Name                     3386 non-null   object 
 2   Continent                3386 non-null   object 
 3   Country                  3386 non-null   object 
 4   Star Rating              3386 non-null   float64
 5   Elevation Change (m)     3386 non-null   int64  
 6   Base Elevation (m)       3386 non-null   int64  
 7   Max Elevation (m)        3386 non-null   int64  
 8   Total Piste Length (km)  3386 non-null   float64
 9   Blue Piste Length (km)   3386 non-null   float64
 10  Red Piste Length (km)    3386 non-null   float64
 11  Black Piste Length (km)  3386 non-null   float64
 12  Ski Lifts                3386 non-null   int64  
 13  Ski Pass Cost            3386 non-null   float64
 14  Currency                

## Visualisation

Visualise the current values to identify any further issues.

In [38]:
import plotly.express as px

In [39]:
# Create a violin plot of the Country against the ski pass cost.
# No obvious outliers are observed
fig = px.violin(df_ski_data, y="Cost in Euros", x="Country", box=True, points="all", title="Country vs Ski Pass Cost")
fig.update_layout(xaxis_type="category", xaxis={'categoryorder':'mean ascending'})

In [40]:
# Look at Star Rating against the Cost in Euros
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ski_data["Star Rating"], 
    y=df_ski_data["Cost in Euros"], 
    name="Star Rating vs Cost in Euros", mode="markers"))

fig.update_layout(title="Star Rating vs Cost in Euros", xaxis_title="Star Rating",
    yaxis_title="Cost in Euros")

fig.show()

The trend is mixed, but there is clearly a corellation. Albeit one that does not apply equally.

In [41]:
# Look at Total Piste Length against the Cost in Euros
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ski_data["Total Piste Length (km)"], 
    y=df_ski_data["Cost in Euros"], 
    name="Piste Length vs Cost in Euros", mode="markers"))

fig.update_layout(title="Piste Length vs Cost in Euros", xaxis_title="Total Piste Length (km)",
    yaxis_title="Cost in Euros")

fig.show()

There are certainly outliers, but the cost does appear to be correlated with length. However there are diminishing returns as the length increases.

In [42]:
# Save the clean data for reference
df_ski_data.to_csv("results/ski_resort_data_LR_ready.csv")

In [43]:
# Review the correlations in the data
df_ski_data.corr()

,ID,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Cost in Euros
ID,1.000000,-0.420154,-0.335211,-0.219292,-0.317198,-0.236765,-0.201485,-0.229001,-0.207221,-0.243469,-0.059378,-0.332355
Star Rating,-0.420154,1.000000,0.793367,0.326989,0.613969,0.704085,0.604322,0.683865,0.601575,0.642469,0.059581,0.689632
Elevation Change (m),-0.335211,0.793367,1.000000,0.382946,0.753076,0.656918,0.577205,0.650491,0.513445,0.595452,0.047257,0.506267
Base Elevation (m),-0.219292,0.326989,0.382946,1.000000,0.896167,0.290492,0.206484,0.281174,0.325301,0.167100,0.036781,0.304493
Max Elevation (m),-0.317198,0.613969,0.753076,0.896167,1.000000,0.522435,0.424313,0.512711,0.478312,0.405027,0.048895,0.460044
Total Piste Length (km),-0.236765,0.704085,0.656918,0.290492,0.522435,1.000000,0.904155,0.952280,0.811182,0.863867,-0.002276,0.543253
Blue Piste Length (km),-0.201485,0.604322,0.577205,0.206484,0.424313,0.904155,1.000000,0.777003,0.577759,0.881443,-0.005641,0.364552
Red Piste Length (km),-0.229001,0.683865,0.650491,0.281174,0.512711,0.952280,0.777003,1.000000,0.737355,0.805617,-0.010116,0.518917
Black Piste Length (km),-0.207221,0.601575,0.513445,0.325301,0.478312,0.811182,0.577759,0.737355,1.000000,0.557972,0.020015,0.659781
Ski Lifts,-0.243469,0.642469,0.595452,0.167100,0.405027,0.863867,0.881443,0.805617,0.557972,1.000000,0.034705,0.385119


In [44]:
px.imshow(df_ski_data.corr(), color_continuous_scale='Agsunset', title="Correlation heatmap of Ski Resort Data")

Ski lifts highly correlated with piste lengths as expected.
The black piste length actually appears to be the strongest correlated to the cost.


## Train/Test Data

The aim is to use the features of the ski resorts to estimate the ski pass costs. The first step shall be to split the data into training and test sets. 
The sklearn module can be used to facilitate this.

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
# Stasmodels package doesn't accept spaces or brackets in the column names, these will need to be replaced
df_ski_data.columns = df_ski_data.columns.str.replace(" ", "_", regex=False)
df_ski_data.columns = df_ski_data.columns.str.replace("(", "", regex=False)
df_ski_data.columns = df_ski_data.columns.str.replace(")", "", regex=False)  
df_ski_data.columns

Index(['ID', 'Name', 'Continent', 'Country', 'Star_Rating', 'Elevation_Change_m', 'Base_Elevation_m', 'Max_Elevation_m', 'Total_Piste_Length_km', 'Blue_Piste_Length_km', 'Red_Piste_Length_km', 'Black_Piste_Length_km', 'Ski_Lifts', 'Ski_Pass_Cost', 'Currency', 'Cost_in_Euros'], dtype='object')

In [47]:
# Because we will be using statsmodel for the linear regression we actually want to keep the labels with the feature data
X_train, X_test, y_train, y_test = train_test_split(df_ski_data, df_ski_data['Cost_in_Euros'], test_size=0.3)

In [48]:
# Lets do a few checks to see how well split the data is
round(X_train['Country'].value_counts()/len(X_train['Country'])*100, 2)

USA                       11.98
Germany                   11.27
Austria                    9.87
Switzerland                8.06
Japan                      7.97
France                     6.75
Italy                      6.62
Czech Republic             6.29
Canada                     5.70
Norway                     4.60
Sweden                     3.71
Poland                     3.16
Slovakia                   2.49
Finland                    1.81
Spain                      1.05
Slovenia                   0.97
New Zealand                0.80
United Kingdom             0.68
Romania                    0.59
Russia                     0.51
Chile                      0.46
Argentina                  0.42
Greece                     0.38
Ukraine                    0.38
Bosnia and Herzegovina     0.34
Iceland                    0.30
Australia                  0.25
Netherlands                0.25
Hungary                    0.21
China                      0.21
Belgium                    0.17
Serbia  

In [49]:
round(X_test['Country'].value_counts()/len(X_test['Country'])*100, 2)

Germany                   12.30
USA                       11.81
Austria                    9.35
Japan                      8.76
Switzerland                7.97
France                     6.20
Czech Republic             6.10
Italy                      6.00
Canada                     5.71
Sweden                     5.41
Norway                     4.04
Slovakia                   3.35
Poland                     3.35
Finland                    1.67
Slovenia                   0.98
Russia                     0.98
Romania                    0.59
Bosnia and Herzegovina     0.49
Spain                      0.49
United Kingdom             0.49
New Zealand                0.49
China                      0.39
South Korea                0.39
Ukraine                    0.30
Bulgaria                   0.30
Chile                      0.30
Turkey                     0.30
Iceland                    0.20
Hungary                    0.20
Israel                     0.10
Australia                  0.10
Belgium 

Looking at the percentages the split appears even.

## Linear Regression

In [50]:
import statsmodels.formula.api as smf

Strong correlations observed to piste lengths, star rating and elevation change as anticipated.

In [51]:
# Select columns to fit to model, categorical variables will be identified automatically
# Exclude Max Elevation as discussed earlier.
features = ["Continent", "Country", "Star_Rating", "Elevation_Change_m", "Base_Elevation_m", "Total_Piste_Length_km",
             "Blue_Piste_Length_km", "Red_Piste_Length_km", "Black_Piste_Length_km", "Ski_Lifts"]

In [52]:
resort_model = smf.ols("Cost_in_Euros ~ " + " + ".join(features), X_train).fit()
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     122.0
Date:                Wed, 07 Apr 2021   Prob (F-statistic):               0.00
Time:                        11:21:43   Log-Likelihood:                -8782.7
No. Observations:                2370   AIC:                         1.769e+04
Df Residuals:                    2309   BIC:                         1.804e+04
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                            -14.5909      2.502     -5.833      0.000     -19.496      -9.685
Continent[T.Asia]                    -11.8717      2.655     -4.471      0.000     -17.079      -6.665
Continent[T.Australia and Oceania]    19.7451      1.955     10.102      0.000      15.912      23.578
Continent[T.Europe]                  -14.0867      5.203     -2.707      0.007     -24.290      -3.883
Continent[T.North America]             6.5919      2.692      2.449      0.014       1.313      11.871
Continent[T.South America]            -8.8777      2.817     -3.151      0.002     -14.402      -3.353
Country[T.Argentina]                 -17.2205      3.623     -4.753      0.000     -24.325     -10.116
Country[T.Armenia]                   -25.4673      9.451     -2.695      0.007     -44.001      -6.934
Country[T.Australia]                  19.8360      2.882      6.883      0.000      14.185      25.487
Country[T.Austria]                    12.9194      5.861      2.204      0.028       1.427      24.412
Country[T.Azerbaijan]                -15.6736      6.869     -2.282      0.023     -29.143      -2.204
Country[T.Belgium]                    16.2239      7.697      2.108      0.035       1.131      31.317
Country[T.Bosnia and Herzegovina]      0.5924      6.813      0.087      0.931     -12.768      13.953
Country[T.Brazil]                     -2.3921      7.489     -0.319      0.749     -17.078      12.293
Country[T.Bulgaria]                   -0.4610      8.198     -0.056      0.955     -16.538      15.616
Country[T.Canada]                     -6.1962      2.736     -2.264      0.024     -11.562      -0.830
Country[T.Chile]                      10.7349      3.556      3.019      0.003       3.762      17.708
Country[T.China]                      45.4741      4.688      9.701      0.000      36.282      54.666
Country[T.Croatia]                     7.1876     11.563      0.622      0.534     -15.487      29.862
Country[T.Cyprus]                      1.2585      9.439      0.133      0.894     -17.251      19.769
Country[T.Czech Republic]              8.7724      5.901      1.487      0.137      -2.800      20.345
Country[T.Denmark]                    16.6277      8.238      2.018      0.044       0.473      32.782
Country[T.Estonia]                    23.3935      9.176      2.549      0.011       5.400      41.387
Country[T.Finland]                    25.4478      6.056      4.202      0.000      13.572      37.324
Country[T.France]                      4.3158      5.858      0.737      0.461      -7.172      15.804
Country[T.Georgia]                   -29.6157      5.129     -5.775      0.000     -39.673     -19.559
Country[T.Germany]                    11.5920      5.880      1.971      0.049       0.061      

The initial attempt lead to a reasonable value of Rsquared being achieved of 0.758. The categorical data in Continent and Country has been identified correctly. However not all the countries are significant. It might be worth combining them into smaller groups.

The number of ski lifts does not appear to be significant. This is likely a consequence of this significance being captured in the piste lengths.

Also use the ANOVA1 test to confirm that the country variable is significant (ANOVA2 is unnecessary since its only one variable). This is not a question for the Continent variable since every entry had a low p-value.

In [ ]:
resort_model_mean = smf.ols("Cost_in_Euros ~ 1", X_train).fit()
resort_model_country = smf.ols("Cost_in_Euros ~ Country", X_train).fit()
# Look at the residuals - They clearly demonstrate a big impact
print("Residuals when fitting the mean: ", resort_model_mean.ssr, "\nResiduals when fitting Country: ", resort_model_country.ssr)

In [ ]:
import statsmodels.api as sm

In [ ]:
sm.stats.anova_lm(resort_model_country, typ=1)

In [64]:
# Have a look at the countries with high p values
resort_model.pvalues[resort_model.pvalues > 0.05]

Country[T.Bosnia and Herzegovina]    0.994385
Country[T.Brazil]                    0.722015
Country[T.Bulgaria]                  0.894523
Country[T.Croatia]                   0.577996
Country[T.Cyprus]                    0.895003
Country[T.Czech Republic]            0.168057
Country[T.Denmark]                   0.053879
Country[T.France]                    0.530036
Country[T.Germany]                   0.064011
Country[T.Greece]                    0.440047
Country[T.Greenland]                 0.648357
Country[T.Hungary]                   0.218637
Country[T.Iceland]                   0.087062
Country[T.Italy]                     0.251087
Country[T.Lesotho]                   0.474002
Country[T.Liechtenstein]             0.359419
Country[T.Lithuania]                 0.107990
Country[T.Montenegro]                0.401576
Country[T.New Zealand]               0.954666
Country[T.North Macedonia]           0.614394
Country[T.Poland]                    0.133251
Country[T.Romania]                

In [161]:
# Some surprising results in the country breakdown with France, Germany (Just about) and Italy appearing despite being common skiing nations

# One way to separate them would be based on each country's cost of living, since this would be reflected in the cost
# An alternative way would be based on the number of resorts per country. Competition could lead to reduced costs, on the other hand resorts can compete across borders so it would be arbitrary.

# To get a better feel for things, lets try removing the country (index 1) as a feature
features_exclude_country = features[:1] + features[2:]
resort_model = smf.ols("Cost_in_Euros ~ " + " + ".join(features_exclude_country), X_train).fit()
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     423.6
Date:                Wed, 07 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:04:44   Log-Likelihood:                -9232.7
No. Observations:                2370   AIC:                         1.849e+04
Df Residuals:                    2358   BIC:                         1.856e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                            -16.1509     12.123     -1.332      0.183     -39.924       7.623
Continent[T.Asia]                     -1.9662     12.014     -0.164      0.870     -25.525      21.593
Continent[T.Australia and Oceania]    23.0611     12.279      1.878      0.060      -1.017      47.139
Continent[T.Europe]                   -2.3675     11.988     -0.197      0.843     -25.876      21.141
Continent[T.North America]             8.9085     12.012      0.742      0.458     -14.646      32.463
Continent[T.South America]            -8.4991     12.276     -0.692      0.489     -32.572      15.574
Star_Rating                           17.4275      0.838     20.804      0.000      15.785      19.070
Elevation_Change_m                     0.0008      0.001      0.617      0.537      -0.002       0.003
Base_Elevation_m                       0.0015      0.000      2.992      0.003       0.001       0.002
Total_Piste_Length_km                  0.1910      0.010     18.619      0.000       0.171       0.211
Blue_Piste_Length_km                  -0.4048      0.026    -15.442      0.000      -0.456      -0.353
Red_Piste_Length_km                   -0.2413      0.033     -7.294      0.000      -0.306      -0.176
Black_Piste_Length_km                  0.8371      0.041     20.425      0.000       0.757       0.918
==============================================================================
Omnibus:                      619.984   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4263.623
Skew:                           1.054   Prob(JB):                         0.00
Kurtosis:                       9.223   Cond. No.                     1.91e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.92e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [124]:
# Removing Countries not only makes the model worse it also reduces the weight of other features which is an interesting side effect

# To categorise the countries by cost of living we shall need data!
# We shall use the GDP per Capita from the World Bank
df_gdp = pd.read_excel("country_gdp_per_capita.xlsx", usecols="A,B")
df_gdp.head()

C:\Users\awk09_000\miniconda3\envs\aicore\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning:

Workbook contains no default style, apply openpyxl's default



,,2018
0,Afghanistan,493.8
1,Albania,"5,284.4"
2,Algeria,"4,153.7"
3,American Samoa,"11,466.7"
4,Andorra,"41,793.1"


In [126]:
df_gdp.columns = ["country", "gdp_per_capita"]
df_gdp.head()

,country,gdp_per_capita
0,Afghanistan,493.8
1,Albania,"5,284.4"
2,Algeria,"4,153.7"
3,American Samoa,"11,466.7"
4,Andorra,"41,793.1"


In [127]:
# Need to check if the gdp data is available for every country
ski_countries = set(df_ski_data["Country"])
countries = set(df_gdp["country"])

# Check the ski resort countries are a subset of the PPP stat countries
ski_countries < countries

False

In [128]:
# False! This means some countries are missing or have different names
# Get the countries in common
in_both = ski_countries & countries
# Now check for the countries not included
ski_countries - in_both

{'Iran', 'Kyrgyzstan', 'Russia', 'Slovakia', 'South Korea', 'USA'}

In [129]:
# This looks like a naming issue
# Rename these 1 by 1...
iran_index = df_gdp[df_gdp["country"] == 'Iran, Islamic Rep.'].index[0]
df_gdp.loc[iran_index, "country"] = 'Iran'

kyrg_index = df_gdp[df_gdp["country"] == 'Kyrgyz Republic'].index[0]
df_gdp.loc[kyrg_index, "country"] = 'Kyrgyzstan'

russia_index = df_gdp[df_gdp["country"] == 'Russian Federation'].index[0]
df_gdp.loc[russia_index, "country"] = 'Russia'

slovak_index = df_gdp[df_gdp["country"] == 'Slovak Republic'].index[0]
df_gdp.loc[slovak_index, "country"] = 'Slovakia'

SK_index = df_gdp[df_gdp["country"] == 'Korea, Rep.'].index[0]
df_gdp.loc[SK_index, "country"] = 'South Korea'

usa_index = df_gdp[df_gdp["country"] == 'United States'].index[0]
df_gdp.loc[usa_index, "country"] = 'USA'


In [130]:
# Check again
ski_countries = set(df_ski_data["Country"])
countries = set(df_gdp["country"])

# Check the ski resort countries are a subset of the PPP stat countries
ski_countries < countries

True

In [142]:
# Now we need to create a new column in the original data set (not the train/test split) with the gdp values for each country
# We'll have to loop through the countries since not all shall be represented in the original data set.

gdp = []

for country in df_ski_data["Country"]:
    # Create a catch for the countries not included
    gdp.append(df_gdp['gdp_per_capita'][df_gdp["country"] == country].values[0])
    
df_ski_data["gdp"] = gdp

In [148]:
# Now modify the values so they become numerical
df_ski_data["gdp"] = df_ski_data["gdp"].str.replace(",","").astype(float)

In [149]:
# Lets look at the statistics
df_ski_data["gdp"].describe()

count      3386.000000
mean      47182.219876
std       19320.418014
min        1221.900000
25%       34615.800000
50%       47810.500000
75%       54589.100000
max      181402.800000
Name: gdp, dtype: float64

In [152]:
# Use the quartiles to split into categories
# Use the following: vlow_income, low_income, med_income, high_income

gdp_cat=[]

for gdp in df_ski_data["gdp"]:
    if gdp < 35000:
        gdp_cat.append('vlow_income')
    elif gdp < 48000: # Not including > 35000 since its executed in order
        gdp_cat.append('low_income')
    elif gdp < 55000:
        gdp_cat.append('med_income')
    else:
        gdp_cat.append('high_income')

df_ski_data["gdp_cat"] = gdp_cat    
df_ski_data["gdp_cat"].unique()

array(['med_income', 'vlow_income', 'high_income', 'low_income'],
      dtype=object)

In [154]:
# First re-split the data
# Because we will be using statsmodel for the linear regression we actually want to keep the labels with the feature data
X_train, X_test, y_train, y_test = train_test_split(df_ski_data, df_ski_data['Cost_in_Euros'], test_size=0.3)

In [174]:
# Lets try a model with this new category
# Recreate the features list
features = ["Continent", "gdp_cat", "Star_Rating", "Elevation_Change_m", "Base_Elevation_m", "Total_Piste_Length_km",
             "Blue_Piste_Length_km", "Red_Piste_Length_km", "Black_Piste_Length_km"]

resort_model = smf.ols("Cost_in_Euros ~ " + " + ".join(features), X_train).fit()
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     427.5
Date:                Wed, 07 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:21:25   Log-Likelihood:                -9026.6
No. Observations:                2370   AIC:                         1.808e+04
Df Residuals:                    2355   BIC:                         1.817e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -2.9253     11.141     -0.263      0.793     -24.772      18.922
Continent[T.Asia]                     -2.4096     11.037     -0.218      0.827     -24.052      19.233
Continent[T.Australia and Oceania]    19.3881     11.281      1.719      0.086      -2.733      41.510
Continent[T.Europe]                   -6.3122     11.002     -0.574      0.566     -27.886      15.262
Continent[T.North America]             0.3610     11.040      0.033      0.974     -21.289      22.011
Continent[T.South America]            -7.7163     11.262     -0.685      0.493     -29.800      14.368
gdp_cat[T.low_income]                -11.7240      0.659    -17.779      0.000     -13.017     -10.431
gdp_cat[T.med_income]                 -4.9301      0.812     -6.071      0.000      -6.523      -3.338
gdp_cat[T.vlow_income]               -12.7778      0.731    -17.491      0.000     -14.210     -11.345
Star_Rating                           17.2215      0.781     22.037      0.000      15.689      18.754
Elevation_Change_m                    -0.0004      0.001     -0.339      0.734      -0.003       0.002
Base_Elevation_m                       0.0015      0.000      3.365      0.001       0.001       0.002
Total_Piste_Length_km                  0.1919      0.009     20.313      0.000       0.173       0.210
Blue_Piste_Length_km                  -0.3966      0.024    -16.402      0.000      -0.444      -0.349
Red_Piste_Length_km                   -0.2420      0.030     -7.969      0.000      -0.302      -0.182
Black_Piste_Length_km                  0.8305      0.038     22.085      0.000       0.757       0.904
==============================================================================
Omnibus:                      664.121   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6389.948
Skew:                           1.040   Prob(JB):                         0.00
Kurtosis:                      10.771   Cond. No.                     2.09e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.56e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [185]:
# Things have got worse and several features have pvalues > 0.05
# On a positive note there is a clear correlation with the cost
# Lets try dropping the Continent feature
features_ex_locations = features[1:]

resort_model = smf.ols("Cost_in_Euros ~ " + " + ".join(features_ex_locations), X_train).fit()
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.698
Method:                 Least Squares   F-statistic:                     608.8
Date:                Wed, 07 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:29:39   Log-Likelihood:                -9102.5
No. Observations:                2370   AIC:                         1.822e+04
Df Residuals:                    2360   BIC:                         1.828e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -12.5960      1.688     -7.461      0.000     -15.906      -9.286
gdp_cat[T.low_income]    -12.1227      0.621    -19.507      0.000     -13.341     -10.904
gdp_cat[T.med_income]     -7.6988      0.765    -10.059      0.000      -9.200      -6.198
gdp_cat[T.vlow_income]   -15.3475      0.678    -22.642      0.000     -16.677     -14.018
Star_Rating               20.2679      0.721     28.094      0.000      18.853      21.683
Elevation_Change_m        -0.0042      0.001     -3.771      0.000      -0.006      -0.002
Base_Elevation_m           0.0016      0.000      3.355      0.001       0.001       0.002
Total_Piste_Length_km      0.1995      0.010     20.948      0.000       0.181       0.218
Blue_Piste_Length_km      -0.4572      0.024    -18.946      0.000      -0.505      -0.410
Red_Piste_Length_km       -0.2505      0.031     -8.069      0.000      -0.311      -0.190
Black_Piste_Length_km      0.9072      0.037     24.450      0.000       0.834       0.980
==============================================================================
Omnibus:                      683.940   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5339.469
Skew:                           1.143   Prob(JB):                         0.00
Kurtosis:                       9.989   Cond. No.                     3.72e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.08e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [184]:
# Result has gotten worse again, but all pvalues are now below 0.05
# Try combining Continent and gdp_cat as an alternative
feature_string = " + ".join(features)
feature_string = feature_string.replace("+", "*", 1)

resort_model_cont_gdp_combo = smf.ols("Cost_in_Euros ~ " + feature_string, X_train).fit()
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.721
Method:                 Least Squares   F-statistic:                     340.8
Date:                Wed, 07 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:29:35   Log-Likelihood:                -9004.0
No. Observations:                2370   AIC:                         1.805e+04
Df Residuals:                    2351   BIC:                         1.816e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    -1.8822      4.176     -0.451      0.652     -10.071       6.306
Continent[T.Asia]                                            -4.4194      5.045     -0.876      0.381     -14.312       5.474
Continent[T.Australia and Oceania]                           18.5806      5.563      3.340      0.001       7.673      29.489
Continent[T.Europe]                                          -8.4916      3.918     -2.168      0.030     -16.174      -0.809
Continent[T.North America]                                    2.0570      3.964      0.519      0.604      -5.716       9.830
Continent[T.South America]                                   -4.1721      5.583     -0.747      0.455     -15.120       6.775
gdp_cat[T.low_income]                                        -9.4192      1.400     -6.730      0.000     -12.164      -6.675
gdp_cat[T.med_income]                                        -3.8563      3.641     -1.059      0.290     -10.996       3.283
gdp_cat[T.vlow_income]                                      -12.4001      7.166     -1.730      0.084     -26.453       1.652
Continent[T.Asia]:gdp_cat[T.low_income]                       0.0612      1.262      0.048      0.961      -2.414       2.537
Continent[T.Australia and Oceania]:gdp_cat[T.low_income]     -1.4624      4.395     -0.333      0.739     -10.080       7.155
Continent[T.Europe]:gdp_cat[T.low_income]                     0.2320      1.519      0.153      0.879      -2.746       3.210
Continent[T.North America]:gdp_cat[T.low_income]             -8.2499      1.644     -5.017      0.000     -11.474      -5.025
Continent[T.South America]:gdp_cat[T.low_income]             1.9e-15   1.44e-14      0.132      0.895   -2.64e-14    3.02e-14
Continent[T.Asia]:gdp_cat[T.med_income]                    2.672e-15   8.77e-15      0.305      0.761   -1.45e-14    1.99e-14
Continent[T.Australia and Oceania]:gdp_cat[T.med_income]  -1.945e-15   2.59e-15     -0.751      0.452   -7.02e-15    3.13e-15
Continent[T.Europe]:gdp_cat[T.med_income]                     0.6310      3.677      0.172      0.864      -6.580       7.842
Continent[T.North America]:gdp_cat[T.med_income]             -4.4873      7.268     -0.617      0.537     -18.740       9.765
Continent[T.South America]:gdp_cat[T.med_income]          -1.312e-18   1.88e-15     -0.001      0.999   -3.69e-15    3.68e-15
Continent[T.Asia]:gdp_cat[T.vlow_income]                     -4.4805      6.070     -0.738      0.461     -16.384       7.423
Continent[T.Australia and Oceania]:gdp_cat[T.vlow_income]          0          0        nan        nan           0           0
Continent[T.Europe]:gdp_cat[T.vlow_income]                 

In [ ]:
# Results got better, but p values are poor. Probably overfitting.

In [186]:
# Look at the residuals over the resorts
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=X_train.index, 
    y=resort_model.resid, 
    name="Model Residuals", mode="markers"))

fig.update_layout(title="Model Residuals", xaxis_title="Resorts",
    yaxis_title="Residuals")

fig.show()

In [187]:
# Look at the residuals over the Star Ratings
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=X_train["Star_Rating"], 
    y=resort_model.resid, 
    name="Model Residuals", mode="markers"))

fig.update_layout(title="Model Residuals", xaxis_title="Star Ratings",
    yaxis_title="Residuals")

fig.show()

In [188]:
# Plot the fitted values against the actual values

#X_train["fitted_costs"] = resort_model.fittedvalues

fig = go.Figure()
# Note that we still plot against the original Year variable
fig.add_trace(go.Scatter(
    x=X_train["Cost_in_Euros"], y=resort_model.fittedvalues, name="Actual vs Fitted Costs", mode="markers"))
#fig.add_trace(go.Scatter(
   # x=X_train.index, y=X_train["fitted_costs"], name="Predicted Costs", mode="markers"))
fig.update_layout(title="Predicted Costs vs Actual Costs", xaxis_title="Actual Costs in Euros",
    yaxis_title="Predicted Costs in Euros")
fig.show()

In [189]:
print(abs(resort_model.resid).mean())
np.sqrt((resort_model.resid ** 2).mean())

7.679282151001749


11.265788655197301

In [191]:
# Lets predict from the test set

# Plot the predicted values against the actual values

#X_train["fitted_costs"] = resort_model.fittedvalues

fig = go.Figure()
# Note that we still plot against the original Year variable
fig.add_trace(go.Scatter(
    x=X_test["Cost_in_Euros"], y=resort_model.predict(X_test), name="Actual vs Predicted Costs", mode="markers"))

fig.update_layout(title="Predicted Costs vs Actual Costs", xaxis_title="Actual Costs in Euros",
    yaxis_title="Predicted Costs in Euros")
fig.show()